In [2]:
import gc
import os
import time

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from glob import glob
from tqdm import tqdm
from keras import backend as K
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping
from keras.preprocessing import image                  
from sklearn.utils import shuffle
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Flatten, Dense
from keras.models import Sequential, Model
from keras.layers import BatchNormalization
from keras import regularizers, applications, optimizers, initializers
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7

In [3]:
def binary_accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)))

def precision_threshold(threshold = 0.5):
    def precision(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(y_pred)
        precision_ratio = true_positives / (predicted_positives + K.epsilon())
        return precision_ratio
    return precision

def recall_threshold(threshold = 0.5):
    def recall(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.clip(y_true, 0, 1))
        recall_ratio = true_positives / (possible_positives + K.epsilon())
        return recall_ratio
    return recall

def fbeta_score_threshold(beta = 1, threshold = 0.5):
    def fbeta_score(y_true, y_pred):
        threshold_value = threshold
        beta_value = beta
        p = precision_threshold(threshold_value)(y_true, y_pred)
        r = recall_threshold(threshold_value)(y_true, y_pred)
        bb = beta_value ** 2
        fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
        return fbeta_score
    return fbeta_score

def calculate_cm(y_true, y_pred):
    fp = np.sum((y_pred == 1) & (y_true == 0))
    tp = np.sum((y_pred == 1) & (y_true == 1))
    fn = np.sum((y_pred == 0) & (y_true == 1))
    tn = np.sum((y_pred == 0) & (y_true == 0))
    return tp, fp, fn, tn

def calculate_recall(tp, fp, fn, tn):
    return (tp)/(tp + fn)

def calculate_fallout(tp, fp, fn, tn):
    return (fp)/(fp + tn)

def calculate_fpr_tpr(y_true, y_pred):
    tp, fp, fn, tn = calculate_cm(y_true, y_pred)
    tpr = calculate_recall(tp, fp, fn, tn)
    fpr = calculate_fallout(tp, fp, fn, tn)
    return fpr, tpr

In [4]:
log = CSVLogger('saved_models/log_pretrained_CNN.csv')
checkpointer = ModelCheckpoint(filepath='saved_models/pretrainedVGG.best.from_scratch.hdf5', verbose=1, save_best_only=True)

In [5]:
diseases = [
    'Cardiomegaly','Emphysema','Effusion',
    'Hernia','Nodule','Pneumothorax',
    'Atelectasis','Pleural_Thickening',
    'Mass','Edema','Consolidation',
    'Infiltration','Fibrosis','Pneumonia'
]

# dataset_df = pd.read_csv('/kaggle/input/data/Data_Entry_2017.csv')
dataset_df = pd.read_csv('./dataset_information/Data_Entry_2017.csv')

In [6]:
# Applying One Hot Encoding to Labels
for disease in diseases:
    dataset_df[disease] = dataset_df['Finding Labels'].apply(lambda x: 1 if disease in x else 0)

In [7]:
image_labels = dataset_df[diseases].to_numpy()
image_paths = {
    # os.path.basename(x): x for x in glob(os.path.join('..', 'input', 'data', 'images*', 'images', '*.png'))
    os.path.basename(x): x for x in glob(os.path.join('.', 'images', '*.png'))
}

print(f"Samples Found: {len(image_paths)}")

Samples Found: 112120


In [8]:
# Storing path to each image against image name in the dataframe
dataset_df['Image Path'] = dataset_df['Image Index'].map(image_paths.get)

In [9]:
images_list = dataset_df['Image Path'].tolist()

labelB = (dataset_df[diseases].sum(axis = 1) > 0).tolist()
labelB = np.array(labelB, dtype = int)

In [10]:
def read_image_to_tensor(path, shape):
    # Loads RGB image to PIL format
    img = image.load_img(path, target_size = shape)
    
    # Convert PIL image to 3D tensor of specific shape
    # and normalizes it by dividing each pixel by 255
    normalized_image_tensor = image.img_to_array(img) / 255
    
    # Convert 3D tensor to 4D tensor with specific shape 
    # (1, shape, 3) and return it
    return np.expand_dims(normalized_image_tensor, axis = 0)

In [11]:
def image_to_array(paths, shape):
    images_arrays = [read_image_to_tensor(path, shape) for path in tqdm(paths, desc = "Progress", ncols = 100)]
    return np.vstack(images_arrays)

In [12]:
# Defining some hyper-parameters
IMAGE_SHAPE = (70, 70)
EPOCHS = 10
BATCH_SIZE = 32

In [13]:
# Samples in Training Set: 70%
# Samples in Validation Set: 10%

# Storing labels of samples for each set
train_labels = labelB[ : 78484][ : , np.newaxis]
valid_labels = labelB[78484 : 89696][ : , np.newaxis]

# Storing arrays of samples for each set
training_samples = image_to_array(images_list[ : 78484], shape = IMAGE_SHAPE)
validation_samples = image_to_array(images_list[78484 : 89696], shape = IMAGE_SHAPE)

Progress: 100%|███████████████████████████████████████████████| 11212/11212 [02:51<00:00, 65.33it/s]


In [14]:
# Creating a model with EfficientNet as base.
e_net = EfficientNetB0(
    weights = 'imagenet',
    include_top = False,
    input_shape = training_samples.shape[1 : ]
)

custom_classifier = Sequential()
custom_classifier.add(GlobalAveragePooling2D(input_shape = e_net.output_shape[1 : ]))
custom_classifier.add(Dropout(0.2))
custom_classifier.add(Dense(256, activation = 'relu'))
custom_classifier.add(Dropout(0.2))
custom_classifier.add(Dense(512, activation = 'relu'))
custom_classifier.add(Dropout(0.2))
custom_classifier.add(Dense(256, activation = 'relu'))
custom_classifier.add(Dropout(0.2))
custom_classifier.add(Dense(50, activation = 'relu'))
custom_classifier.add(Dropout(0.2))
custom_classifier.add(Dense(1, activation = 'sigmoid'))

model = Model(inputs = e_net.input, outputs = custom_classifier(e_net.output))
# model.summary()

In [15]:
# Defining 2 optimizers to test the model with.

SGD_optimizer = tf.keras.optimizers.SGD(
    learning_rate = 1e-4, 
    decay = 1e-6, 
    momentum = 0.9, 
    nesterov = True
)
adam_optimizer = tf.keras.optimizers.Adam(
    learning_rate = 0.001,
    beta_1 = 0.9,
    beta_2 = 0.999,
)

In [16]:
# Defining object for augmentation 

train_datagen = ImageDataGenerator(
    featurewise_center = False,  # set input mean to 0 over the dataset
    samplewise_center = False,  # set each sample mean to 0
    featurewise_std_normalization = False,  # divide inputs by std of the dataset
    samplewise_std_normalization = False,  # divide each input by its std
    zca_whitening = False,  # apply ZCA whitening
    rotation_range = 10,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range = 0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range = 0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip = True,  # randomly flip images
    vertical_flip = False 
)

In [17]:
# Compiling model with loss function, optimizer and metrics

model.compile(
    optimizer = SGD_optimizer,
    loss = 'binary_crossentropy',
    metrics = [
        'accuracy',
        tf.keras.metrics.FalseNegatives(),
        tf.keras.metrics.TrueNegatives(),
        tf.keras.metrics.FalsePositives(),
        tf.keras.metrics.TrueNegatives(),
        precision_threshold(threshold = 0.5), 
        recall_threshold(threshold = 0.5), 
        fbeta_score_threshold(beta=0.5, threshold = 0.5)
    ]
)

In [18]:
%%timeit -n1 -r1

history = model.fit_generator(
    train_datagen.flow(
        training_samples, 
        train_labels, 
        batch_size = BATCH_SIZE
    ),
    steps_per_epoch = len(training_samples) // BATCH_SIZE,
    validation_data = (validation_samples, valid_labels),
    validation_steps = len(validation_samples) // BATCH_SIZE,
    epochs = EPOCHS,
    callbacks = [checkpointer], 
    verbose = 1
)

C:\Users\m7irt\anaconda3\envs\tf-gpu\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
2452/2452 [==============================] - 341s 132ms/step - loss: 0.6949 - accuracy: 0.5228 - false_negatives: 25551.0000 - true_negatives: 31235.0000 - false_positives: 11889.0000 - true_negatives_1: 31235.0000 - precision: 0.4570 - recall: 0.2771 - fbeta_score: 0.3711 - val_loss: 0.7043 - val_accuracy: 0.4752 - val_false_negatives: 5884.0000 - val_true_negatives: 5328.0000 - val_false_positives: 0.0000e+00 - val_true_negatives_1: 5328.0000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_fbeta_score: 0.0000e+00

Epoch 00001: val_loss improved from inf to 0.70427, saving model to saved_models\pretrainedVGG.best.from_scratch.hdf5


C:\Users\m7irt\anaconda3\envs\tf-gpu\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/10
2452/2452 [==============================] - 331s 135ms/step - loss: 0.6877 - accuracy: 0.5462 - false_negatives: 30012.0000 - true_negatives: 37540.0000 - false_positives: 5587.0000 - true_negatives_1: 37540.0000 - precision: 0.4812 - recall: 0.1502 - fbeta_score: 0.3159
Epoch 3/10
2452/2452 [==============================] - 335s 137ms/step - loss: 0.6841 - accuracy: 0.5542 - false_negatives: 29024.0000 - true_negatives: 37177.0000 - false_positives: 5951.0000 - true_negatives_1: 37177.0000 - precision: 0.5166 - recall: 0.1784 - fbeta_score: 0.3556
Epoch 4/10
2452/2452 [==============================] - 333s 136ms/step - loss: 0.6797 - accuracy: 0.5648 - false_negatives: 27177.0000 - true_negatives: 36155.0000 - false_positives: 6967.0000 - true_negatives_1: 36155.0000 - precision: 0.5401 - recall: 0.2325 - fbeta_score: 0.4110
Epoch 5/10
2452/2452 [==============================] - 333s 136ms/step - loss: 0.6717 - accuracy: 0.5860 - false_negatives: 23868.0000 - true_negat

In [19]:
# Freeing Memory with Python's Garbase Collector
del training_samples
del validation_samples
del train_labels
del valid_labels
gc.collect()

1075

In [20]:
# Samples in Testing Set: 20%
test_labels = labelB[89696 : ][ : , np.newaxis]
test_samples = image_to_array(images_list[89696 : ], shape = IMAGE_SHAPE)

prediction = model.predict(test_samples)

Progress: 100%|███████████████████████████████████████████████| 22424/22424 [06:04<00:00, 61.60it/s]


In [21]:
# Evaluation of Trained Model
testing_predictions = []
for val in prediction:
    if val[0] < 0.5:
        testing_predictions.append([0])
    else:
        testing_predictions.append([1])

testing_predictions = np.array(testing_predictions)

TP, FP, FN, TN = calculate_cm(test_labels, testing_predictions)
FPR, TPR = calculate_fpr_tpr(test_labels, testing_predictions)

threshold = 0.5
beta = 0.5

accuracy = K.eval(binary_accuracy(K.variable(value=test_labels), K.variable(value=prediction)))
precision = K.eval(precision_threshold(threshold = threshold)(K.variable(value=test_labels),K.variable(value=prediction)))
recall = K.eval(recall_threshold(threshold = threshold)(K.variable(value=test_labels),K.variable(value=prediction)))
f1_score = K.eval(fbeta_score_threshold(beta = beta, threshold = threshold)(K.variable(value=test_labels),K.variable(value=prediction)))

print (f"Accuracy: {accuracy} \nRecall: {recall} \nSpecificity: {TN / (TN + FP)}\nPrecision: {precision} \nF1-Score: {f1_score}\n")
print (f"True Positives: {TP} \nFalse Positives: {FP} \nFalse Negatives: {FN} \nTrue Negatives: {TN}\n")
print (f"False Positve Rate: {FPR * 100} % \nTrue Positive Rate: {TPR * 100} %")

Accuracy: 0.46780234575271606 
Recall: 1.0 
Specificity: 0.0004187955440154117
Precision: 0.46768367290496826 
F1-Score: 0.5234072804450989

True Positives: 10485 
False Positives: 11934 
False Negatives: 0 
True Negatives: 5

False Positve Rate: 99.95812044559847 % 
True Positive Rate: 100.0 %
